## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-08-21-14-50-18 +0000,startribune,Live at the State Fair: Follow along as we try...,https://www.startribune.com/mn-state-fair-food...
1,2025-08-21-14-50-04 +0000,startribune,Appeals court throws out massive civil fraud p...,https://www.startribune.com/appeals-court-thro...
2,2025-08-21-14-49-26 +0000,nypost,At least 600 CDC employees getting final notic...,https://nypost.com/2025/08/21/us-news/at-least...
3,2025-08-21-14-49-00 +0000,wsj,"Robust U.S. Activity Data Supports Dollar, Tre...",https://www.wsj.com/finance/currencies/indian-...
4,2025-08-21-14-48-26 +0000,nyt,Walmart’s Earnings Show ‘Momentum’ as Tariffs ...,https://www.nytimes.com/2025/08/21/business/wa...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2317/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
20,trump,52
6,new,25
58,ukraine,18
60,russia,14
214,israel,12


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
259,2025-08-20-20-36-51 +0000,nypost,Kremlin’s new demands threaten to derail peace...,https://nypost.com/2025/08/20/us-news/kremlins...,126
52,2025-08-21-13-22-32 +0000,nypost,Russian missiles hit US-owned factory in Ukrai...,https://nypost.com/2025/08/21/world-news/russi...,101
193,2025-08-21-01-00-00 +0000,wsj,President Trump and his allies are waging a ca...,https://www.wsj.com/economy/central-banking/fe...,100
7,2025-08-21-14-43-51 +0000,nypost,Trump defends Ukraine’s attack on Russia with ...,https://nypost.com/2025/08/21/us-news/trump-de...,99
202,2025-08-21-00-19-00 +0000,wsj,President Trump has ordered the Pentagon to se...,https://www.wsj.com/politics/national-security...,98


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
259,126,2025-08-20-20-36-51 +0000,nypost,Kremlin’s new demands threaten to derail peace...,https://nypost.com/2025/08/20/us-news/kremlins...
193,48,2025-08-21-01-00-00 +0000,wsj,President Trump and his allies are waging a ca...,https://www.wsj.com/economy/central-banking/fe...
119,48,2025-08-21-09-39-40 +0000,bbc,Palestinians flee Gaza City districts as Israe...,https://www.bbc.com/news/articles/clyr7l0z9edo...
67,46,2025-08-21-12-30-03 +0000,nypost,US keeping lofty EU car tariffs until levies o...,https://nypost.com/2025/08/21/us-news/us-keepi...
250,43,2025-08-20-20-57-46 +0000,nypost,"JD Vance, Stephen Miller mock ‘old, white’ pro...",https://nypost.com/2025/08/20/us-news/jd-vance...
26,41,2025-08-21-14-13-25 +0000,cbc,"Judge Frank Caprio, known online as the 'nices...",https://www.cbc.ca/news/world/judge-frank-capr...
202,40,2025-08-21-00-19-00 +0000,wsj,President Trump has ordered the Pentagon to se...,https://www.wsj.com/politics/national-security...
154,38,2025-08-21-05-56-59 +0000,nypost,Screaming man who tried to run from federal of...,https://nypost.com/2025/08/21/us-news/dc-migra...
224,37,2025-08-20-22-21-26 +0000,nyt,Ousted Fire Chief in L.A. Claims Mayor Bass Re...,https://www.nytimes.com/2025/08/20/us/los-ange...
236,34,2025-08-20-21-45-00 +0000,wsj,President Trump has called on a Biden-appointe...,https://www.wsj.com/economy/central-banking/fe...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
